---
title: "Einführung in Maschinelle Lernverfahren"
jupyter: ir
---


In [ ]:
#| include: false
req_pkg <- c(
  "riskCommunicator", "data.table", "splitTools",
  "kdry", "kableExtra"
)
for (r in req_pkg) {
  if (!(r %in% installed.packages()[, "Package"])) {
    install.packages(r)
  }
}


In [ ]:
#| echo: true
dataset_full <- riskCommunicator::framingham |>
  data.table::data.table() # Daten einlesen
# Subset: Basisuntersuchung
dataset <- dataset_full[get("PERIOD") == 1, ]

# Relevante Spalten definieren
use_cols <- c("SEX", "TOTCHOL", "AGE", "SYSBP",
"CURSMOKE", "CIGPDAY", "BMI", "DIABETES",
"HYPERTEN")
# Relevante Spalten filtern, fehlende Werte entfernen
dataset <- dataset[
  , .SD, .SDcols = use_cols
] |>na.omit()

# Transformieren der katgeorialen Variablen
# "SEX" "CURSMOKE" "CIGPDAY" "DIABETES" "HYPERTEN"
cat_vars <- use_cols[c(1, 5, 8, 9)]
# Datentyp "factor" ändern
dataset[, (cat_vars) := lapply(
  X = .SD,
  FUN = factor),
  .SDcols = cat_vars
]


In [ ]:
#| echo: false
#| include: false
rm(dataset_full, cat_vars, use_cols, r, req_pkg)


In [ ]:
# Übersicht über den Datensatz --> n=4332 Beobachtungen
str(dataset)


In [ ]:
#| echo: true
# Ausgabe der ersten 10 Zeilen
dataset[1:10, ]


In [ ]:
#| echo: true
# Die Funktion erzeugt eine Liste mit Indices für die jeweiligen Datensets
# Das stratifizierte Splitten anhand der Zielvariable "HYPERTEN" soll deren
# gleichmäßige Verteilung in den Teildatensätzen sicherstellen.
data_splits <- splitTools::partition(
  y = dataset$HYPERTEN,
  p = c(train = 0.7, validation = 0.15, test = 0.15),
  type = "stratified",
  seed = 123
)
sapply(data_splits, length)  # Verteilung der Beobachtungen auf die Teildatensätze


In [ ]:
#| include: false
# Initialisierung einer leeren Tabelle und Sammeln der kontinuierlichen / diskreten Variablen in je einem Vektor
results_table <- data.table::data.table()
num_vars <- dataset[, sapply(.SD, is.numeric), .SDcols = colnames(dataset)]
# Diskrete Variablen ohne 'CIGPDAY' (zu viele Ausprägungen für Visualisierung)
cat_vars <- dataset[, sapply(.SD, is.factor), .SDcols = setdiff(colnames(dataset), "CIGPDAY")]

# Helper-Funktion definieren für Counts der diskreten Variablen
level_counts <- function(X, cols) {
  lvls <- levels(cols[, get(X)])
  out <- sapply(
    X = lvls,
    FUN = function(x) {
      kdry::rep_frac_pct(
        count = cols[get(X) == x, .N],
        count_reference = cols[, .N]
      )
    },
    simplify = TRUE
  )
  names(out) <- paste(X, names(out), sep = "=")
  out <- out |>
    cbind() |>
    data.table::data.table(keep.rownames = TRUE)
  return(out)
}

# Berechnung von Mittelwert/Standardabweichung (kontinuierlichen Variablen) bzw.
# Summe/relative Häufigkeit (diskrete Variablen) für Teildatensätze
for (sp in names(data_splits)) {
  num.cols <- dataset[data_splits[[sp]], names(num_vars)[num_vars], with = FALSE]
  cat.cols <- dataset[data_splits[[sp]], names(cat_vars)[cat_vars], with = FALSE]

  add_col_num <- sapply(X = num.cols, FUN = kdry::rep_mean_sd) |>
    cbind() |>
    data.table::data.table(keep.rownames = TRUE)
  colnames(add_col_num)[2] <- "out"

  add_col_cat <- lapply(X = names(cat.cols), FUN = level_counts, cols = cat.cols) |>
    data.table::rbindlist()

  results_table <- cbind(results_table, rbind(add_col_num, add_col_cat))
}

# Redundate Spalten entfernen
results_table <- results_table[, c(1, 2, 4, 6)];

# Spaltenbeschriftungen
colnames(results_table) <- c("Variable", names(data_splits))
# jeweiliges N an die Split-Bezeichnungen in den Spaltennamen ergänzen
split_colnames <- sapply(X = names(data_splits), FUN = function(x) {paste0(x, " (n=", length(data_splits[[x]]), ")")})
colnames(results_table)[2:4] <- split_colnames


In [ ]:
#| echo: true
knitr::kable(results_table, caption = "Train-Validation-Test Split") |>
  kableExtra::kable_styling(font_size = "70%")


In [ ]:
#| echo: false
#| include: false
rm(num_vars, cat_vars, level_counts, num.cols, cat.cols,
  add_col_num, add_col_cat, results_table, sp, split_colnames)


In [ ]:
#| echo: true
# Teildatensätze für Regressions- und Klassifizierungs-Beispiele

# Regression: Zielvariable "SYSBP" --> Entfernen von "HYPERTEN"
dataset_reg <- dataset[
  , .SD, .SDcols = setdiff(colnames(dataset), "HYPERTEN")
]

# Klassifikation: Zielvariable "HYPERTEN" --> Entfernen von "SYSBP"
dataset_cls <- dataset[
  , .SD, .SDcols = setdiff(colnames(dataset), "SYSBP")
]


In [ ]:
#| echo: true
#| out-width: 100%
#| fig-align: center
boxplot(dataset$SYSBP ~ dataset$HYPERTEN)